# Product Segmentation

In [ ]:
#!pip install yellowbrick

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_file = "https://raw.githubusercontent.com/manaranjanp/ISB_MLUL/main/clustering/beer.csv"

In [ ]:
beer_df = pd.read_csv( data_file )

In [ ]:
beer_df

In [ ]:
sn.kdeplot(beer_df.calories);

In [ ]:
sn.kdeplot(beer_df.sodium);

In [ ]:
sn.scatterplot(data = beer_df,
               x = 'alcohol',
               y = 'calories');

## Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_beer_df = scaler.fit_transform( beer_df[['calories', 
                                                'sodium', 
                                                'alcohol', 
                                                'cost']] )

In [ ]:
scaled_beer_df = pd.DataFrame(scaled_beer_df, 
                              columns = ['calories', 
                                         'sodium', 
                                         'alcohol', 
                                         'cost'])

In [ ]:
sn.kdeplot(scaled_beer_df.sodium);
sn.kdeplot(scaled_beer_df.calories);

## How many clusters exist?

#### Using Dendrogram

In [ ]:
cmap = sn.cubehelix_palette(as_cmap=True, rot=-.3, light=1)
sn.clustermap(scaled_beer_df, 
              cmap=cmap, 
              linewidths=.2,               
              figsize = (6,6) );

In [ ]:
beer_df.iloc[[9,15]]

In [ ]:
beer_df.iloc[[16,14]]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
cluster_range = range( 1, 10 )
cluster_errors = []

for num_clusters in cluster_range:
  clusters = KMeans( num_clusters )
  clusters.fit( scaled_beer_df )
  cluster_errors.append( clusters.inertia_ )

plt.figure(figsize=(8,4))
plt.plot( cluster_range, cluster_errors, marker = "o" );

## Evaluating the quality of cluster using Silhouette score 

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

fig, ax = plt.subplots(2, 2, figsize=(15,8))
num_clusters = [2, 3, 4, 5]
for i, k in enumerate(num_clusters):
    km = KMeans(n_clusters=k, 
                random_state=42)
    q, mod = divmod(i, 2)
    visualizer = SilhouetteVisualizer(km, 
                                      colors='yellowbrick', 
                                      ax=ax[q-1][mod])
    visualizer.fit(scaled_beer_df) 

#### Rescaling the dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaled_beer_df = scaler.fit_transform( beer_df[['calories', 
                                                'sodium', 
                                                'alcohol', 
                                                'cost']] )

#### Creating 3 Clusters

We will set k to 3 for running *KMeans* algorithm and create a new column *clusterid* in *beer_df* to capture the cluster number it is assigned to.

In [ ]:
k = 4

clusters = KMeans( k, random_state = 42 )
clusters.fit( scaled_beer_df )
beer_df["clusterid"] = clusters.labels_

In [ ]:
clusters.labels_

## Understanding the clusters

In [ ]:
sn.violinplot(data = beer_df,
              y = 'calories',
              x = 'clusterid');

In [ ]:
sn.violinplot(data = beer_df,
              y = 'cost',
              x = 'clusterid');

In [ ]:
sn.scatterplot(data = beer_df,
               x = 'cost',
               y = 'calories',
               hue = 'clusterid');

#### Cluster 0

In [ ]:
beer_df[beer_df.clusterid == 0]

#### Cluster 1

In [ ]:
beer_df[beer_df.clusterid == 1]

#### Cluster 2

In [ ]:
beer_df[beer_df.clusterid == 2]

#### Cluster 3

In [ ]:
beer_df[beer_df.clusterid == 3]

# Evaluating Cluster Performane

## Calinski-Harabasz Index

In [ ]:
from sklearn.metrics import calinski_harabasz_score

In [ ]:
cluster_range = range( 3, 6 )
ch_scores = []

for num_clusters in cluster_range:
  clusters = KMeans( num_clusters )
  clusters.fit( scaled_beer_df )
  print(f"n_cluster: {num_clusters} - CH Score: {calinski_harabasz_score(scaled_beer_df, clusters.labels_)}" )

## Davies-Bouldin Index

In [ ]:
from sklearn.metrics import davies_bouldin_score

In [ ]:
cluster_range = range( 3, 6 )
ch_scores = []

for num_clusters in cluster_range:
  clusters = KMeans( num_clusters )
  clusters.fit( scaled_beer_df )
  print(f"n_cluster: {num_clusters} - CH Score: {davies_bouldin_score(scaled_beer_df, clusters.labels_)}" )

# Hierarchical Clustering

Cluster method:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

In [ ]:
cmap = sn.cubehelix_palette(as_cmap=True, rot=-.3, light=1)
sn.clustermap(scaled_beer_df, 
              cmap=cmap, 
              linewidths=.2,               
              figsize = (6,6) );

In [ ]:
from sklearn.cluster import AgglomerativeClustering

Try different metric and linkage

- metric: “euclidean”, “manhattan”, “cosine”, "minkowski"
- linkage: single, complete, ward, average

In [ ]:
h_cluster = AgglomerativeClustering(n_clusters = 4,
                                    metric = 'euclidean',
                                    linkage = 'ward',
                                    compute_distances = True)

In [ ]:
#h_cluster = AgglomerativeClustering(n_clusters = 4,
#                                    metric = 'manhattan',
#                                    linkage = 'complete')

In [ ]:
h_cluster.fit(scaled_beer_df)

In [ ]:
h_cluster.labels_

In [ ]:
beer_df['hcluster_clusterid'] = h_cluster.labels_

In [ ]:
beer_df.sort_values('hcluster_clusterid')

In [ ]:
h_cluster.n_leaves_

In [ ]:
h_cluster.children_

In [ ]:
h_cluster.distances_

# HDBScan Clustering

In [ ]:
from sklearn.cluster import HDBSCAN

In [ ]:
hdbscan = HDBSCAN(min_cluster_size=2)

In [ ]:
hdbscan.fit(scaled_beer_df)

In [ ]:
hdbscan.labels_

In [ ]:
beer_df['hdbscan_clusterid'] = hdbscan.labels_

In [ ]:
beer_df[beer_df.hdbscan_clusterid == 0]

In [ ]:
beer_df[beer_df.hdbscan_clusterid == 1]

In [ ]:
beer_df[beer_df.hdbscan_clusterid == 2]

In [ ]:
beer_df[beer_df.hdbscan_clusterid == 3]

In [ ]:
beer_df[beer_df.hdbscan_clusterid == -1]